# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [3]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [7]:
ruta_principal = "/content/drive/MyDrive/IronHack/"
salaries = pd.read_csv(ruta_principal + "Current_Employee_Names__Salaries__and_Position_Titles.csv")

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [8]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [9]:
salaries["Salary or Hourly"].value_counts()

,count
Salary or Hourly,
Salary,25161
Hourly,8022


In [10]:
salaries[salaries["Salary or Hourly"]=="Hourly"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 8022 entries, 4 to 33175
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               8022 non-null   object 
 1   Job Titles         8022 non-null   object 
 2   Department         8022 non-null   object 
 3   Full or Part-Time  8022 non-null   object 
 4   Salary or Hourly   8022 non-null   object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      0 non-null      float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 564.0+ KB


In [11]:
salaries[salaries["Salary or Hourly"]=="Salary"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 25161 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               25161 non-null  object 
 1   Job Titles         25161 non-null  object 
 2   Department         25161 non-null  object 
 3   Full or Part-Time  25161 non-null  object 
 4   Salary or Hourly   25161 non-null  object 
 5   Typical Hours      0 non-null      float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        0 non-null      float64
dtypes: float64(3), object(5)
memory usage: 1.7+ MB


In [12]:
print("Con las dos celdas anteriores descubrimos que no hay valores nulos individuales.")
print("• Los nulos provienen de que HourlyRate y Typical Hours son nulos para los individuos que cobran salario")
print("• Los nulos provienen de que Annual Salary son nulos para los individuos que cobran por horas")

Con las dos celdas anteriores descubrimos que no hay valores nulos individuales.
• Los nulos provienen de que HourlyRate y Typical Hours son nulos para los individuos que cobran salario
• Los nulos provienen de que Annual Salary son nulos para los individuos que cobran por horas


# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [19]:
from scipy.stats import ttest_1samp

# Se seleccionan los salarios por hora
salarios = salaries[salaries["Salary or Hourly"]=="Hourly"]["Hourly Rate"]

# Se determinan las hipótesis nula y alternativa
print("Hipótesis nula (𝐻0): El salario promedio por hora es igual a $30/hora (μ=30).")
print("Hipótesis alternativa (𝐻1): El salario promedio por hora es diferente de $30/hora (𝜇≠30).\n")

# Prueba t
resultado = ttest_1samp(salarios, 30)

# Resultado de la prueba
print(f"El estadístico t es {resultado.statistic} y el p valor es {resultado.pvalue}.")

# La prueba t se realiza con un nivel de significancia del 5% (equivalente a un intervalo de confianza del 95% para una prueba bilateral).
if resultado.pvalue < 1-0.95:
    print("Rechazamos la hipótesis nula. El salario promedio por hora es significativamente diferente de $30/hora.")
else:
    print("No rechazamos la hipótesis nula. No hay suficiente evidencia para concluir que el salario promedio por hora sea significativamente diferente de $30/hora.")

Hipótesis nula (𝐻0): El salario promedio por hora es igual a $30/hora (μ=30).
Hipótesis alternativa (𝐻1): El salario promedio por hora es diferente de $30/hora (𝜇≠30).

El estadístico t es 20.6198057854942 y el p valor es 4.3230240486229894e-92.
Rechazamos la hipótesis nula. El salario promedio por hora es significativamente diferente de $30/hora.


El **estadístico t** se calcula como:

$$t = \frac{\bar{X} - \mu}{\sigma / \sqrt{N}} $$

Siendo:

* $\bar{X}$: la media de la muestra,
* $\mu$: el valor de referencia (en este caso, $30/hora),

* $\sigma$: la desviación estándar de la muestra,
* $N$: el tamaño de la muestra.



# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [22]:
from scipy import stats

# Se seleccionan los salarios por hora
salarios = salaries[salaries["Salary or Hourly"]=="Hourly"]["Hourly Rate"]

# Parámetros de la muestra
media = np.mean(salarios)
sem = stats.sem(salarios)
N = len(salarios)
grados_libertad = N - 1

# Calcular intervalo de confianza
intervalo_confianza = stats.t.interval(
    confidence=0.95,  # Nivel de confianza
    df=grados_libertad,
    loc=media,
    scale=sem  # Error estándar de la media
)

intervalo_confianza

(32.52345834488425, 33.05365708767623)

In [24]:
print(f"En un 95% de confianza podemos decir que el salario por hora se encuentra entre {round(intervalo_confianza[0],2)}$ y {round(intervalo_confianza[1],2)}$.")

En un 95% de confianza podemos decir que el salario por hora se encuentra entre 32.52$ y 33.05$.


El **Error Estándar de la Media** o *Standard Error of the Mean (SEM)* se calcula como:

$$SEM = \frac{\sigma}{\sqrt{N}} $$

Siendo:
* $\sigma$: la desviación estándar de la muestra.
* $N$: el tamaño de la muestra.


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [33]:
from statsmodels.stats.proportion import proportions_ztest

# Se seleccionan los salarios por hora
salarios = salaries[salaries["Salary or Hourly"]=="Hourly"]["Hourly Rate"]

# Datos y parámetros
total = len(salaries) # Número total trabajadores en Chicago (población observada)
muestra = len(salarios)   # Tamaño total de la muestra por hora (muestra de los trabajadores por hora)
diferente = 0.25

# Prueba z de proporciones
statistic, p_value = proportions_ztest(muestra, total, value=diferente)

# Resultado de la prueba
print(f"El estadístico z es {statistic} y el p valor es {p_value}.")

# La prueba t se realiza con un nivel de significancia del 5% (equivalente a un intervalo de confianza del 95% para una prueba bilateral).
if p_value < 1-0.95:
    print(f"Rechazamos la hipótesis nula. Concluyendo que la proporción de trabajadores por hora en Chicago es significativamente diferente de {int(diferente*100)}%")
else:
    print(f"No rechazamos la hipótesis nula. No hay suficiente evidencia para concluir que la proporción sea diferente de {int(diferente*100)}%.")


El estadístico z es -3.5099964213703005 y el p valor es 0.0004481127249057967.
Rechazamos la hipótesis nula. Concluyendo que la proporción de trabajadores por hora en Chicago es significativamente diferente de 25%


Es el estadístico z, que mide cuántas desviaciones estándar está la proporción muestral del valor de referencia (0.25)